In [ ]:
# !pip install spacy datasets -qq

In [2]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ovzr56xa
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-ovzr56xa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 636 kB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 36.5 MB/s 
     |████████████████████████████████| 895 kB 62.7 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Created wheel for transformers: filename=transformers-4.11.0.dev0-py3-none-any.whl size=2833295 sha256=00de26d329e99588e68a9d59b6c2d5b818c307713273391798df1a9193453557
  Stored in directory: /tmp/pip-ephem-wheel-cache-7jgg60u0/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully unin

In [1]:
import os
import torch
import pandas as pd
import spacy
from spacy.tokenizer import Tokenizer

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)

# Library setups
from google.colab import drive
drive.mount('/content/drive')

nlp = spacy.load("en_core_web_sm")

ModuleNotFoundError: ignored

In [ ]:
# model = BERTEmotionModel()

In [ ]:
%cd drive/MyDrive/Allen_NLP_hackathon

[Errno 2] No such file or directory: 'drive/MyDrive/Allen_NLP_hackathon'
/content/drive/.shortcut-targets-by-id/18wIy9U4zK_LdoQmnfHm1GL3Xzxy8Ki6V/Allen_NLP_hackathon


In [ ]:
# given list of sentences 
def get_word_embedding(sentence, word, model, tokenizer):
  sentence = sentence.lower()
  word = word.lower()
  tokenized_text = tokenizer.tokenize(sentence)
  word_index = tokenized_text.index(word)
  ids = tokenizer.convert_tokens_to_ids(tokenized_text)
  ids = torch.as_tensor([ids, ])
  model_output = model.forward(input_ids=ids,output_hidden_states=True)
  last_state = model_output.hidden_states[-1]
  
  # Collapsing the tensor into 1-dimension
  token_embeddings = torch.squeeze(last_state, dim=0)
  token_embeddings = list(token_embeddings)

  word_embedding = token_embeddings[word_index].detach().numpy()

  return word_embedding


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('Models/anger')
tokenizer = AutoTokenizer.from_pretrained('Models/anger')

In [ ]:
word = 'test'
sentence = 'This is a test sentence!'

In [ ]:
emb = get_word_embedding(sentence, word, model, tokenizer)

In [ ]:
import pandas as pd
from scipy.spatial.distance import cosine

In [ ]:
texts = [
  "Girl",
  "The girl walked to the bank.",
  "This girl's favorite class was Chemistry.",
  "The girl with the long black hair is named Jessica.",
  "I'm a girl, don't mistake me for a boy!"
]

In [ ]:
target_word_embeddings = [get_word_embedding(sentence, "Girl", model, tokenizer) for sentence in texts]

In [ ]:
def gen_cosine_df(texts, target_word_embeddings):
  list_of_distances = []
  for text1, embed1 in zip(texts, target_word_embeddings):
      for text2, embed2 in zip(texts, target_word_embeddings):
          cos_dist = 1 - cosine(embed1, embed2)
          list_of_distances.append([text1, text2, cos_dist])

  distances_df = pd.DataFrame(list_of_distances, columns=['text1', 'text2', 'cosine'])
  return distances_df

In [ ]:
distances_df = gen_cosine_df(texts, target_word_embeddings)
distances_df

,text1,text2,cosine
0,Girl,Girl,1.000000
1,Girl,The girl walked to the bank.,0.996245
2,Girl,This girl's favorite class was Chemistry.,0.993681
3,Girl,The girl with the long black hair is named Jes...,0.995054
4,Girl,"I'm a girl, don't mistake me for a boy!",0.991203
5,The girl walked to the bank.,Girl,0.996245
6,The girl walked to the bank.,The girl walked to the bank.,1.000000
7,The girl walked to the bank.,This girl's favorite class was Chemistry.,0.998299
8,The girl walked to the bank.,The girl with the long black hair is named Jes...,0.998938
9,The girl walked to the bank.,"I'm a girl, don't mistake me for a boy!",0.995956


## Calculate WEAT test and significance

In [ ]:
import numpy as np

In [ ]:
# text with girl replaced with boy
boy_text = [
  "Boy",
  "The boy walked to the bank.",
  "This boy's favorite class was Chemistry.",
  "The boy with the long black hair is named Jessica.",
  "I'm a boy, don't mistake me for a boy!"
]

In [ ]:
# target sets with girl, and girl replaced with boy
target_girl = target_word_embeddings
target_boy = [get_word_embedding(sentence, "Boy", model, tokenizer) for sentence in boy_text]

In [ ]:
# attribute sets -- will need to change
# maybe change to anger attributes next time 
attr_caucasian = ["caucasian", "caucasian", "white", "america", "america", "europe"]
attr_asian = ["asian", "asia", "china", "asia"]

# turn this into statistical test 

In [ ]:
# measures association of word embedding w
# with attribute sets A and B
def s(w, A, B, model, tokenizer):
    # mean_cos_A = np.mean([cosine(w, a) for a in A]) # might need to do attr A
    # mean_cos_B = np.mean([cosine(w, b) for b in B])

    embds_A = [get_word_embedding(sent_A, w, model, tokenizer) for sent_A in A]
    mean_cos_A = df_A.cosine.mean()

    embds_B = [get_word_embedding(sent_B, w, model, tokenizer) for sent_B in B]
    mean_cos_A = df_B.cosine.mean()
    return mean_cos_A - mean_cos_B

s_girl = s(target_girl, attr_caucasian, attr_asian, model, tokenizer)
s_girl

AttributeError: ignored

In [ ]:
# WEAT test statistic
# target sets X, Y
# attribute sets A, B
def test_state(X, Y, A, B):
  X_strength = np.sum([s(x, A, B) for x in X] )
  Y_strength = np.sum([s(y, A, B) for y in Y] )
  return X_strength - Y_strength


P value
- one sided p-value of the permutation test is
$P[s(X_i, Y_i, A, B) > s(X, Y, A, B)]$

Formally, consider a single set of target words
W and two sets of attribute words A, B. There is a
property $p_w$ associated with each word $w \in W$.

The null hypothesis is that there is no difference between the two sets of target words in terms of their relative similarity to the two sets of attribute words. The permutation test measures the (un)likelihood of the null hypothesis by computing the probability that a random permutation of the attribute words would product the observed (or greater) difference in sample means. 
- null hypothesis: shuffled data sets should look like real data 
- otherwise: shuffled data should look different from real data

In [ ]:
# P value for permutation test 
# Pr_i[s(X_i, Y_i, A, B) > S(X, Y, A, B)]

prob = [] # calculate whether s(X_i, Y_i, A, B) > S(X, Y, A, B) for each i 
# for s


In [ ]:
# effect size
# normalized measure of how separated thetwo disttributions are 